In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLkNN
import sklearn.metrics as metrics
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import multilabel_confusion_matrix
import os

In [2]:
# Process label 1 dataset for Hindi
df_l1 = pd.read_csv('train_hi_l1.csv')
df_l1 = df_l1.rename(columns={'key': 'unique_id', 'sentence': 'text'})

# Convert annotator columns to numeric without replacing NaNs (only 5 annotators)
df_l1[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']] = df_l1[
    ['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label_1' based on majority voting while ignoring NaNs
df_l1['label_1'] = (df_l1[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']].mean(axis=1, skipna=True) >= 0.5).astype(int)

# Create proper binary label for task 1 (gendered abuse)
df_l1['binary_label_1'] = df_l1['label_1'].apply(lambda x: 'hate' if x == 1 else 'not_hate')

# Process label 3 dataset for Hindi
df_l3 = pd.read_csv('train_hi_l3.csv')
df_l3 = df_l3.rename(columns={'key': 'unique_id', 'sentence': 'text'})

# Convert annotator columns to numeric without replacing NaNs (only 5 annotators)
df_l3[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']] = df_l3[
    ['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label_3' based on majority voting while ignoring NaNs
df_l3['label_3'] = (df_l3[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']].mean(axis=1, skipna=True) >= 0.5).astype(int)

# Create proper binary label for task 3 (explicit/aggressive)
df_l3['binary_label_3'] = df_l3['label_3'].apply(lambda x: 'explicit' if x == 1 else 'not_explicit')

# Select columns for merging
df_l1_slim = df_l1[['text', 'label_1', 'binary_label_1']]
df_l3_slim = df_l3[['text', 'label_3', 'binary_label_3']]

# Merge the datasets based on text field
merged_df = pd.merge(df_l1_slim, df_l3_slim, on='text', how='inner')

# Add a unique_id column to the merged dataset
merged_df['unique_id'] = [f'id_{i}' for i in range(len(merged_df))]

# Reorder columns
merged_df = merged_df[['unique_id', 'text', 'binary_label_1', 'label_1', 'binary_label_3', 'label_3']]

# Save the merged dataset
merged_df.to_csv('train_hi_task3.csv', index=False)

# Display information
print(f"Final merged dataset shape: {merged_df.shape}")
print(merged_df.head())

# Check label distribution
print("\nLabel distribution:")
print(f"Label 1 (gendered abuse): {merged_df['binary_label_1'].value_counts().to_dict()}")
print(f"Label 3 (explicit/aggressive): {merged_df['binary_label_3'].value_counts().to_dict()}")

Final merged dataset shape: (6196, 6)
  unique_id                                               text binary_label_1  \
0      id_0    भारत में तीन किसान कानून बिल वापस लेने पर भक...       not_hate   
1      id_1    राजस्थान  अजीबोगरीब: गांव की खुशहाली के लिए ...       not_hate   
2      id_2    सलमान ने की राखी के विरोधी की वकालत तो 'आइटम...       not_hate   
3      id_3  !!हर शब्द अमॄतम!!  पुरानी एक कहावत है... टूटी ...       not_hate   
4      id_4  "PM मोदी जी " की माँ के लिए अपशब्द कहने वाला द...           hate   

   label_1 binary_label_3  label_3  
0        0   not_explicit        0  
1        0   not_explicit        0  
2        0   not_explicit        0  
3        0   not_explicit        0  
4        1       explicit        1  

Label distribution:
Label 1 (gendered abuse): {'not_hate': 4426, 'hate': 1770}
Label 3 (explicit/aggressive): {'not_explicit': 3320, 'explicit': 2876}


In [3]:
import nltk
import string
import re

def normalize_text(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F700-\U0001F77F"
                               u"\U0001F780-\U0001F7FF"
                               u"\U0001F800-\U0001F8FF"
                               u"\U0001F900-\U0001F9FF"
                               u"\U0001FA00-\U0001FA6F"
                               u"\U0001FA70-\U0001FAFF"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = text.lower()
    text = re.sub(r'\[.*?\]', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'<.*?>+', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\w*\d\w*', ' ', text)
    text = re.sub(r'<handle replaced>', '', text)
    text = emoji_pattern.sub(r'', text)
    return text

# Load the multi-task dataset
merged_df = pd.read_csv('train_hi_task3.csv')

# Apply text normalization
merged_df['processed_text'] = merged_df['text'].apply(lambda x: normalize_text(x))

# Further processing to remove '...'
merged_df['processed_text'] = merged_df['processed_text'].str.replace('...', '')

# Display samples of processed text
print("\nOriginal vs Processed Text Samples:")
for i in range(3):
    print(f"Original: {merged_df['text'].iloc[i]}")
    print(f"Processed: {merged_df['processed_text'].iloc[i]}")
    print("-" * 50)

# Keep all columns but add processed text
merged_df_final = merged_df[['unique_id', 'text', 'processed_text', 'binary_label_1', 'label_1', 'binary_label_3', 'label_3']]


print(f"\nProcessed dataset shape: {merged_df_final.shape}")
print(merged_df_final.head())


Original vs Processed Text Samples:
Original:   भारत में तीन किसान कानून बिल वापस लेने पर भक्तों की जो हालत हुई अब यह किसी से छुपा नहीं है उसके इलावा जो टट्टी खोर पत्रकार थे उनकी भी हालत खराब हुई है किसी को मुंह दिखाने के लायक नहीं है अब यह लोग यहां तक कि अपने घर में भी मुंह दिखाने के लायक नहीं रहे।
Processed:   भारत में तीन किसान कानून बिल वापस लेने पर भक्तों की जो हालत हुई अब यह किसी से छुपा नहीं है उसके इलावा जो टट्टी खोर पत्रकार थे उनकी भी हालत खराब हुई है किसी को मुंह दिखाने के लायक नहीं है अब यह लोग यहां तक कि अपने घर में भी मुंह दिखाने के लायक नहीं रहे।
--------------------------------------------------
Original:   राजस्थान  अजीबोगरीब: गांव की खुशहाली के लिए रात भर किन्नर करते है यह काम...
Processed:   राजस्थान  अजीबोगरीब  गांव की खुशहाली के लिए रात भर किन्नर करते है यह काम   
--------------------------------------------------
Original:   सलमान ने की राखी के विरोधी की वकालत तो 'आइटम गर्ल' ने दी धमकी! #RakhiSawant #SalmanKhan
Processed:   सलमान ने की राखी के विरोधी की वकालत तो  

In [4]:
# Extract features (processed text)
X = list(merged_df_final['processed_text'])

# Extract labels for both tasks
y_task1 = merged_df_final['label_1'].values
y_task3 = merged_df_final['label_3'].values

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    LSTM, Activation, Dropout, Dense, Flatten,
    Bidirectional, GRU, concatenate, SpatialDropout1D,
    GlobalMaxPooling1D, GlobalAveragePooling1D, Conv1D,
    Embedding, Input, Concatenate
)
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import MeanSquaredError

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

######## Textual Features for Embedding ###################
max_len = 100
max_features = 4479

# Tokenization
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding
X = pad_sequences(X, padding='post', maxlen=max_len)

print(X)  # Check the processed sequences

[[  63    5  896 ...    0    0    0]
 [ 629 1047    3 ...    0    0    0]
 [1783   21    3 ...    0    0    0]
 ...
 [   5  357 3995 ...    0    0    0]
 [   6  201 1216 ...    0    0    0]
 [ 205  205  688 ...    0    0    0]]


In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_task1 = label_encoder.fit_transform(y_task1)
y_task3 = label_encoder.fit_transform(y_task3)

In [7]:
from keras.utils import to_categorical
y_task1 = to_categorical(y_task1, num_classes=2)
y_task3 = to_categorical(y_task3, num_classes=2)

In [8]:
import numpy as np
import json

# Load GloVe embeddings from JSON
with open('glove_embeddings.json', encoding="utf8") as f:
    embeddings_list = json.load(f)

# Convert the list of vectors to a dictionary with word indices as keys
embeddings_dictionary = {str(i): vector for i, vector in enumerate(embeddings_list)}

# Define tokenizer 
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size
word_index = tokenizer.word_index
num_words = min(max_features, vocab_size)  # Limit vocab to max_features

# Get embedding dimension (from first vector in list)
embed_size = len(embeddings_list[0]) if embeddings_list else 0

# Initialize embedding matrix
embedding_matrix = np.zeros((num_words, embed_size))

# Fill embedding matrix with corresponding word vectors
for word, index in word_index.items():
    if index >= max_features:
        continue
    embedding_vector = embeddings_dictionary.get(word) or embeddings_dictionary.get(str(index))
    if embedding_vector is not None:
        embedding_matrix[index] = np.asarray(embedding_vector, dtype=np.float32)

print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (4479, 50)


In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, Embedding, SpatialDropout1D, Conv1D,
    Bidirectional, LSTM, Dense, Dropout,
    GlobalAveragePooling1D
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configure GPU for optimal performance
def configure_gpu():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Enable memory growth for each GPU
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
            # Use mixed precision for better performance
            policy = tf.keras.mixed_precision.Policy('mixed_float16')
            tf.keras.mixed_precision.set_global_policy(policy)
            print('Mixed precision enabled')
        except RuntimeError as e:
            print(e)

configure_gpu()

# Multi-Task Model Definition
def create_multi_task_cnn_bilstm_model(max_len, max_features, embedding_matrix, embed_size=300):
    """
    Creates a multi-task CNN-BiLSTM model architecture for joint prediction of
    gendered abuse (task1) and explicit language (task3)
    """
    # Input layer
    input_layer = Input(shape=(max_len,))
    
    # Embedding layer with pretrained weights (GloVe/FastText)
    embedding_layer = Embedding(
        input_dim=max_features,
        output_dim=embed_size,
        weights=[embedding_matrix],
        input_length=max_len,
        trainable=False  # As per paper, embeddings are non-trainable
    )(input_layer)
    
    # Spatial Dropout to prevent overfitting
    spatial_dropout = SpatialDropout1D(0.2)(embedding_layer)
    
    # CNN Layer - shared representation
    conv_layer = Conv1D(
        filters=64,
        kernel_size=2,
        activation='relu',
        padding='same'
    )(spatial_dropout)
    
    # Bidirectional LSTM Layer - shared representation
    bilstm_layer = Bidirectional(
        LSTM(
            units=128,
            return_sequences=True,
            dropout=0.1,
            recurrent_dropout=0.1
        )
    )(conv_layer)
    
    # Global Average Pooling - shared representation
    gap_layer = GlobalAveragePooling1D()(bilstm_layer)
    
    # Task-specific layers for Task 1 (Gendered Abuse)
    task1_dense = Dense(128, activation='relu', name='task1_dense')(gap_layer)
    task1_dropout = Dropout(0.1)(task1_dense)
    task1_output = Dense(2, activation='softmax', name='task1_output', dtype='float32')(task1_dropout)
    
    # Task-specific layers for Task 3 (Explicit Language)
    task3_dense = Dense(128, activation='relu', name='task3_dense')(gap_layer)
    task3_dropout = Dropout(0.1)(task3_dense)
    task3_output = Dense(2, activation='softmax', name='task3_output', dtype='float32')(task3_dropout)
    
    # Create model with multiple outputs
    model = Model(inputs=input_layer, outputs=[task1_output, task3_output])
    
    return model

# Custom MacroF1Score Metric for multi-task learning
class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes=2, name='macro_f1_score', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.tp = self.add_weight(name='tp', initializer='zeros')
        self.fp = self.add_weight(name='fp', initializer='zeros')
        self.fn = self.add_weight(name='fn', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convert probabilities to predicted class indices
        y_pred = tf.argmax(y_pred, axis=-1)
        
        # Convert one-hot encoded y_true to class indices if needed
        if len(y_true.shape) > 1 and y_true.shape[-1] > 1:
            y_true = tf.argmax(y_true, axis=-1)
        
        # Initialize confusion matrix
        conf_matrix = tf.math.confusion_matrix(
            y_true,
            y_pred,
            num_classes=self.num_classes,
            dtype=tf.float32
        )
        
        # Calculate TP, FP, FN for each class
        diag = tf.linalg.diag_part(conf_matrix)
        row_sum = tf.reduce_sum(conf_matrix, axis=1)
        col_sum = tf.reduce_sum(conf_matrix, axis=0)
        
        tp = diag
        fp = col_sum - diag
        fn = row_sum - diag
        
        # Update the state variables
        self.tp.assign_add(tf.reduce_sum(tp))
        self.fp.assign_add(tf.reduce_sum(fp))
        self.fn.assign_add(tf.reduce_sum(fn))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        # Calculate precision and recall
        precision = self.tp / (self.tp + self.fp + tf.keras.backend.epsilon())
        recall = self.tp / (self.tp + self.fn + tf.keras.backend.epsilon())
        
        # Calculate F1 score
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        
        # Return macro F1 (average of per-class F1 scores)
        return f1

    def reset_states(self):
        self.tp.assign(0.)
        self.fp.assign(0.)
        self.fn.assign(0.)
        self.count.assign(0.)

# Model Training for multi-task learning
def train_and_validate_multi_task_model(model, X_train, y_train_task1, y_train_task3, 
                                         X_val, y_val_task1, y_val_task3, 
                                         batch_size=32, epochs=15, model_dir='models_hi_task3_m1'):
    """
    Trains the multi-task CNN-BiLSTM model with early stopping and model checkpointing
    Returns the best model and training history
    """
    # Create directory for saving models if it doesn't exist
    os.makedirs(model_dir, exist_ok=True)
    
    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        mode='min',
        verbose=1
    )
    
    model_checkpoint = ModelCheckpoint(
        os.path.join(model_dir, 'best_model_hi_task3_m1.h5'),
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=1
    )
    
    # Compile model with Adam optimizer
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'task1_output': 'categorical_crossentropy',
            'task3_output': 'categorical_crossentropy'
        },
        loss_weights={
            'task1_output': 1.0,  # Weight for gendered abuse task
            'task3_output': 1.0   # Weight for explicit language task
        },
        metrics={
            'task1_output': ['accuracy', MacroF1Score(num_classes=2)],
            'task3_output': ['accuracy', MacroF1Score(num_classes=2)]
        }
    )
    
    # Train the model
    history = model.fit(
        X_train, 
        {'task1_output': y_train_task1, 'task3_output': y_train_task3},
        validation_data=(
            X_val, 
            {'task1_output': y_val_task1, 'task3_output': y_val_task3}
        ),
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[early_stopping, model_checkpoint],
        verbose=1
    )
    
    # Load the best model found during training
    best_model = load_model(
        os.path.join(model_dir, 'best_model_hi_task3_m1.h5'), 
        custom_objects={'MacroF1Score': MacroF1Score}
    )
    
    return history, best_model

# Plot Training History for multi-task model
def plot_multi_task_training_history(history, plot_dir='plots_hi_task3_m1'):
    """
    Plots training history for both tasks (accuracy and loss curves)
    Saves plots to specified directory
    """
    os.makedirs(plot_dir, exist_ok=True)
    
    # Task 1 (Gendered Abuse) plots
    plt.figure(figsize=(12, 10))
    
    # Plot task1 accuracy
    plt.subplot(2, 2, 1)
    plt.plot(history.history['task1_output_accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_task1_output_accuracy'], label='Validation Accuracy')
    plt.title('Task 1 (Gendered Abuse) Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Plot task1 loss
    plt.subplot(2, 2, 2)
    plt.plot(history.history['task1_output_loss'], label='Train Loss')
    plt.plot(history.history['val_task1_output_loss'], label='Validation Loss')
    plt.title('Task 1 (Gendered Abuse) Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Task 3 (Explicit Language) plots
    # Plot task3 accuracy
    plt.subplot(2, 2, 3)
    plt.plot(history.history['task3_output_accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_task3_output_accuracy'], label='Validation Accuracy')
    plt.title('Task 3 (Explicit Language) Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    # Plot task3 loss
    plt.subplot(2, 2, 4)
    plt.plot(history.history['task3_output_loss'], label='Train Loss')
    plt.plot(history.history['val_task3_output_loss'], label='Validation Loss')
    plt.title('Task 3 (Explicit Language) Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, 'training_history_hi_task3_m1.png'))
    plt.close()
    
    # Plot combined loss
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Total Train Loss')
    plt.plot(history.history['val_loss'], label='Total Validation Loss')
    plt.title('Multi-Task Model Combined Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, 'combined_loss_hi_task3_m1.png'))
    plt.close()

# Evaluation function for multi-task model
def evaluate_multi_task_validation(model, X_val, y_val_task1, y_val_task3, plot_dir='plots_hi_task3_m1'):
    """
    Evaluates the multi-task model on validation data and saves metrics and plots
    """
    os.makedirs(plot_dir, exist_ok=True)
    
    # Predict probabilities
    y_pred_task1, y_pred_task3 = model.predict(X_val, batch_size=32)
    
    # Convert to class labels
    y_pred_task1_labels = np.argmax(y_pred_task1, axis=1)
    y_true_task1_labels = np.argmax(y_val_task1, axis=1)
    
    y_pred_task3_labels = np.argmax(y_pred_task3, axis=1)
    y_true_task3_labels = np.argmax(y_val_task3, axis=1)
    
    # Task 1 (Gendered Abuse) metrics
    task1_precision = precision_score(y_true_task1_labels, y_pred_task1_labels, average='weighted')
    task1_recall = recall_score(y_true_task1_labels, y_pred_task1_labels, average='weighted')
    task1_weighted_f1 = f1_score(y_true_task1_labels, y_pred_task1_labels, average='weighted')
    task1_macro_f1 = f1_score(y_true_task1_labels, y_pred_task1_labels, average='macro')
    
    # Task 3 (Explicit Language) metrics
    task3_precision = precision_score(y_true_task3_labels, y_pred_task3_labels, average='weighted')
    task3_recall = recall_score(y_true_task3_labels, y_pred_task3_labels, average='weighted')
    task3_weighted_f1 = f1_score(y_true_task3_labels, y_pred_task3_labels, average='weighted')
    task3_macro_f1 = f1_score(y_true_task3_labels, y_pred_task3_labels, average='macro')
    
    # Classification reports
    task1_report = classification_report(y_true_task1_labels, y_pred_task1_labels, 
                                        target_names=['not_hate', 'hate'])
    
    task3_report = classification_report(y_true_task3_labels, y_pred_task3_labels, 
                                        target_names=['not_explicit', 'explicit'])
    
    # Confusion matrices
    task1_conf_matrix = confusion_matrix(y_true_task1_labels, y_pred_task1_labels)
    task3_conf_matrix = confusion_matrix(y_true_task3_labels, y_pred_task3_labels)
    
    # Plot confusion matrices
    # Task 1 confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(task1_conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Hate', 'Hate'],
                yticklabels=['Not Hate', 'Hate'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Task 1 (Gendered Abuse) Confusion Matrix')
    plt.savefig(os.path.join(plot_dir, 'task1_confusion_matrix_hi_task3_m1.png'))
    plt.close()
    
    # Task 3 confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(task3_conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Not Explicit', 'Explicit'],
                yticklabels=['Not Explicit', 'Explicit'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Task 3 (Explicit Language) Confusion Matrix')
    plt.savefig(os.path.join(plot_dir, 'task3_confusion_matrix_hi_task3_m1.png'))
    plt.close()
    
    return {
        'task1': {
            'precision': task1_precision,
            'recall': task1_recall,
            'f1_score_weighted': task1_weighted_f1,
            'f1_score_macro': task1_macro_f1,
            'classification_report': task1_report,
            'confusion_matrix': task1_conf_matrix
        },
        'task3': {
            'precision': task3_precision,
            'recall': task3_recall,
            'f1_score_weighted': task3_weighted_f1,
            'f1_score_macro': task3_macro_f1,
            'classification_report': task3_report,
            'confusion_matrix': task3_conf_matrix
        }
    }

# Main Execution for Training and Validation
if __name__ == "__main__":
    # Split into train (80%) and validation (20%)
    X_train, X_val, y_train_task1, y_val_task1, y_train_task3, y_val_task3 = train_test_split(
        X, y_task1, y_task3, test_size=0.2, random_state=42, stratify=y_task1
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Validation samples: {len(X_val)}")
    
    # Create multi-task model
    embed_size = embedding_matrix.shape[1]
    model = create_multi_task_cnn_bilstm_model(max_len, max_features, embedding_matrix, embed_size)
    
    # Print model summary
    model.summary()
    
    # Train multi-task model
    history, trained_model = train_and_validate_multi_task_model(
        model, X_train, y_train_task1, y_train_task3, 
        X_val, y_val_task1, y_val_task3,
        batch_size=32,
        epochs=15  
    )
    
    # Plot training history
    plot_multi_task_training_history(history)
    
    # Evaluate on validation set
    val_results = evaluate_multi_task_validation(
        trained_model, X_val, y_val_task1, y_val_task3
    )
    
    # Print Task 1 (Gendered Abuse) results
    print("\nTask 1 (Gendered Abuse) Validation Results:")
    print(f"Precision: {val_results['task1']['precision']:.4f}")
    print(f"Recall: {val_results['task1']['recall']:.4f}")
    print(f"weighted F1 Score: {val_results['task1']['f1_score_weighted']:.4f}")
    print(f"macro F1 Score: {val_results['task1']['f1_score_macro']:.4f}")
    print("\nClassification Report:")
    print(val_results['task1']['classification_report'])
    
    # Print Task 3 (Explicit Language) results
    print("\nTask 3 (Explicit Language) Validation Results:")
    print(f"Precision: {val_results['task3']['precision']:.4f}")
    print(f"Recall: {val_results['task3']['recall']:.4f}")
    print(f"weighted F1 Score: {val_results['task3']['f1_score_weighted']:.4f}")
    print(f"macro F1 Score: {val_results['task3']['f1_score_macro']:.4f}")
    print("\nClassification Report:")
    print(val_results['task3']['classification_report'])

Training samples: 4956
Validation samples: 1240


C:\Users\krmri\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 100, 50)           │         223,950 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ spatial_dropout1d             │ (None, 100, 50)           │               0 │ embedding[0][0]            │
│ (SpatialDropout1D)            │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 100, 64)           │           6,464 │ spatial_dropout1d[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, 100, 256)          │         197,632 │ conv1d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d      │ (None, 256)               │               0 │ bidirectional[0][0]        │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ task1_dense (Dense)           │ (None, 128)               │          32,896 │ global_average_pooling1d[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ task3_dense (Dense)           │ (None, 128)               │          32,896 │ global_average_pooling1d[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ task1_dense[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 128)               │               0 │ task3_dense[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ task1_output (Dense)          │ (None, 2)                 │             258 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ task3_output (Dense)          │ (None, 2)                 │             258 │ dropout_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 494,354 (1.89 MB)

...

155/155 ━━━━━━━━━━━━━━━━━━━━ 16s 100ms/step - loss: 1.2604 - task1_output_accuracy: 0.7159 - task1_output_loss: 0.5914 - task1_output_macro_f1_score: 0.7159 - task3_output_accuracy: 0.5913 - task3_output_loss: 0.6690 - task3_output_macro_f1_score: 0.5913 - val_loss: 1.2538 - val_task1_output_accuracy: 0.7145 - val_task1_output_loss: 0.5870 - val_task1_output_macro_f1_score: 0.7145 - val_task3_output_accuracy: 0.6016 - val_task3_output_loss: 0.6668 - val_task3_output_macro_f1_score: 0.6016
Epoch 6/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 1.2586 - task1_output_accuracy: 0.7053 - task1_output_loss: 0.5927 - task1_output_macro_f1_score: 0.7053 - task3_output_accuracy: 0.5930 - task3_output_loss: 0.6659 - task3_output_macro_f1_score: 0.5930 
Epoch 6: val_loss did not improve from 1.25378
155/155 ━━━━━━━━━━━━━━━━━━━━ 16s 101ms/step - loss: 1.2586 - task1_output_accuracy: 0.7054 - task1_output_loss: 0.5927 - task1_output_macro_f1_score: 0.7054 - task3_output_accuracy: 0.5930 - tas

155/155 ━━━━━━━━━━━━━━━━━━━━ 16s 105ms/step - loss: 1.2327 - task1_output_accuracy: 0.7134 - task1_output_loss: 0.5764 - task1_output_macro_f1_score: 0.7134 - task3_output_accuracy: 0.6035 - task3_output_loss: 0.6564 - task3_output_macro_f1_score: 0.6035 - val_loss: 1.2484 - val_task1_output_accuracy: 0.7145 - val_task1_output_loss: 0.5850 - val_task1_output_macro_f1_score: 0.7145 - val_task3_output_accuracy: 0.6040 - val_task3_output_loss: 0.6634 - val_task3_output_macro_f1_score: 0.6040
Epoch 8/15
155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 1.2398 - task1_output_accuracy: 0.7103 - task1_output_loss: 0.5845 - task1_output_macro_f1_score: 0.7103 - task3_output_accuracy: 0.6238 - task3_output_loss: 0.6554 - task3_output_macro_f1_score: 0.6238  
Epoch 8: val_loss did not improve from 1.24843
155/155 ━━━━━━━━━━━━━━━━━━━━ 16s 104ms/step - loss: 1.2398 - task1_output_accuracy: 0.7103 - task1_output_loss: 0.5845 - task1_output_macro_f1_score: 0.7103 - task3_output_accuracy: 0.6237 - ta

39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step  


C:\Users\krmri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krmri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krmri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krmri\anaconda3\Lib\site-packag


Task 1 (Gendered Abuse) Validation Results:
Precision: 0.5105
Recall: 0.7145
weighted F1 Score: 0.5955
macro F1 Score: 0.4167

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.71      1.00      0.83       886
        hate       0.00      0.00      0.00       354

    accuracy                           0.71      1240
   macro avg       0.36      0.50      0.42      1240
weighted avg       0.51      0.71      0.60      1240


Task 3 (Explicit Language) Validation Results:
Precision: 0.6092
Recall: 0.6040
weighted F1 Score: 0.6035
macro F1 Score: 0.6039

Classification Report:
              precision    recall  f1-score   support

not_explicit       0.64      0.56      0.60       652
    explicit       0.57      0.66      0.61       588

    accuracy                           0.60      1240
   macro avg       0.61      0.61      0.60      1240
weighted avg       0.61      0.60      0.60      1240



In [11]:
# Process label 1 dataset for Hindi
df_l1 = pd.read_csv('test_hi_l1.csv', engine='python', on_bad_lines='skip')
df_l1 = df_l1.rename(columns={'key': 'unique_id', 'sentence': 'text'})

# Convert annotator columns to numeric without replacing NaNs (only 5 annotators)
df_l1[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']] = df_l1[
    ['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label_1' based on majority voting while ignoring NaNs
df_l1['label_1'] = (df_l1[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']].mean(axis=1, skipna=True) >= 0.5).astype(int)

# Create proper binary label for task 1 (gendered abuse)
df_l1['binary_label_1'] = df_l1['label_1'].apply(lambda x: 'hate' if x == 1 else 'not_hate')

# Process label 3 dataset for Hindi
df_l3 = pd.read_csv('test_hi_l1.csv', engine='python', on_bad_lines='skip')
df_l3 = df_l3.rename(columns={'key': 'unique_id', 'sentence': 'text'})

# Convert annotator columns to numeric without replacing NaNs (only 5 annotators)
df_l3[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']] = df_l3[
    ['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']
].apply(pd.to_numeric, errors='coerce')

# Compute 'label_3' based on majority voting while ignoring NaNs
df_l3['label_3'] = (df_l3[['hi_a1', 'hi_a2', 'hi_a3', 'hi_a4', 'hi_a5']].mean(axis=1, skipna=True) >= 0.5).astype(int)

# Create proper binary label for task 3 (explicit/aggressive)
df_l3['binary_label_3'] = df_l3['label_3'].apply(lambda x: 'explicit' if x == 1 else 'not_explicit')

# Select columns for merging
df_l1_slim = df_l1[['text', 'label_1', 'binary_label_1']]
df_l3_slim = df_l3[['text', 'label_3', 'binary_label_3']]

# Merge the datasets based on text field
merged_df = pd.merge(df_l1_slim, df_l3_slim, on='text', how='inner')

# Add a unique_id column to the merged dataset
merged_df['unique_id'] = [f'id_{i}' for i in range(len(merged_df))]

# Reorder columns
merged_df = merged_df[['unique_id', 'text', 'binary_label_1', 'label_1', 'binary_label_3', 'label_3']]

# Save the merged dataset
merged_df.to_csv('test_hi_task3.csv', index=False)

# Display information
print(f"Final merged dataset shape: {merged_df.shape}")
print(merged_df.head())

# Check label distribution
print("\nLabel distribution:")
print(f"Label 1 (gendered abuse): {merged_df['binary_label_1'].value_counts().to_dict()}")
print(f"Label 3 (explicit/aggressive): {merged_df['binary_label_3'].value_counts().to_dict()}")

Final merged dataset shape: (1517, 6)
  unique_id                                               text binary_label_1  \
0      id_0  #BandraStation #SharadPawar #Muradabad  अगर अभ...       not_hate   
1      id_1  #ConspiracyAgainstIndia  सुन लो रे देश के गद्द...       not_hate   
2      id_2  #MarathaReservation : महाराष्ट्र में जश्न का म...       not_hate   
3      id_3  #RheaChakraborty aap Mahesh bhatt se madad kiy...           hate   
4      id_4  #SecularMaskOfd हमे पता नहि क्या कहते है ,इतना...       not_hate   

   label_1 binary_label_3  label_3  
0        0   not_explicit        0  
1        0   not_explicit        0  
2        0   not_explicit        0  
3        1       explicit        1  
4        0   not_explicit        0  

Label distribution:
Label 1 (gendered abuse): {'not_hate': 1159, 'hate': 358}
Label 3 (explicit/aggressive): {'not_explicit': 1159, 'explicit': 358}


In [12]:
# Apply text normalization
merged_df['processed_text'] = merged_df['text'].apply(lambda x: normalize_text(x))

# Further processing to remove '...'
merged_df['processed_text'] = merged_df['processed_text'].str.replace('...', '')

# Display samples of processed text
print("\nOriginal vs Processed Text Samples:")
for i in range(3):
    print(f"Original: {merged_df['text'].iloc[i]}")
    print(f"Processed: {merged_df['processed_text'].iloc[i]}")
    print("-" * 50)

# Keep all columns but add processed text
merged_df_final = merged_df[['unique_id', 'text', 'processed_text', 'binary_label_1', 'label_1', 'binary_label_3', 'label_3']]


print(f"\nProcessed dataset shape: {merged_df_final.shape}")
print(merged_df_final.head())

# Extract features (processed text)
X = list(merged_df_final['processed_text'])

# Extract labels for both tasks
y_task1 = merged_df_final['label_1'].values
y_task3 = merged_df_final['label_3'].values
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding
X = pad_sequences(X, padding='post', maxlen=max_len)

y_task1 = label_encoder.fit_transform(y_task1)
y_task3 = label_encoder.fit_transform(y_task3)

y_task1 = to_categorical(y_task1, num_classes=2)
y_task3 = to_categorical(y_task3, num_classes=2)


Original vs Processed Text Samples:
Original: #BandraStation #SharadPawar #Muradabad  अगर अभी आपको होश आ गया हो तो ये मान लीजिये की अब आप भारत के नागरिक नही है , अब आप आने वाले समय के शरिया कानून का पालन करते नजर आएंगे जहां काफिरों के लिए अलग क़ानून होगा और मुसलमानों के लिए अलग।  #गजवायेHind की शुरूआत हो चुकी है #10ईयर और।
Processed:  bandrastation  sharadpawar  muradabad  अगर अभी आपको होश आ गया हो तो ये मान लीजिये की अब आप भारत के नागरिक नही है   अब आप आने वाले समय के शरिया कानून का पालन करते नजर आएंगे जहां काफिरों के लिए अलग क़ानून होगा और मुसलमानों के लिए अलग।   गजवायेhind की शुरूआत हो चुकी है    और।
--------------------------------------------------
Original: #ConspiracyAgainstIndia  सुन लो रे देश के गद्दारों घर के और बाहर वाले भी यदि तुम अभी भी नहीं सुधरे ना तो तुम्हारी गांड में छाता डालकर मोर बना दूंगा
Processed:  conspiracyagainstindia  सुन लो रे देश के गद्दारों घर के और बाहर वाले भी यदि तुम अभी भी नहीं सुधरे ना तो तुम्हारी गांड में छाता डालकर मोर बना दूंगा
--------------------

In [13]:
# Evaluate on validation set
test_results = evaluate_multi_task_validation(
    trained_model, X, y_task1, y_task3
)

# Print Task 1 (Gendered Abuse) results
print("\nTask 1 (Gendered Abuse) Test Results:")
print(f"Precision: {test_results['task1']['precision']:.4f}")
print(f"Recall: {test_results['task1']['recall']:.4f}")
print(f"weighted F1 Score: {test_results['task1']['f1_score_weighted']:.4f}")
print(f"macro F1 Score: {test_results['task1']['f1_score_macro']:.4f}")
print("\nClassification Report:")
print(test_results['task1']['classification_report'])

# Print Task 3 (Explicit Language) results
print("\nTask 3 (Explicit Language) Test Results:")
print(f"Precision: {test_results['task3']['precision']:.4f}")
print(f"Recall: {test_results['task3']['recall']:.4f}")
print(f"weighted F1 Score: {test_results['task3']['f1_score_weighted']:.4f}")
print(f"macro F1 Score: {test_results['task3']['f1_score_macro']:.4f}")
print("\nClassification Report:")
print(test_results['task3']['classification_report'])

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


C:\Users\krmri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krmri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krmri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\krmri\anaconda3\Lib\site-packag


Task 1 (Gendered Abuse) Test Results:
Precision: 0.5837
Recall: 0.7640
weighted F1 Score: 0.6618
macro F1 Score: 0.4331

Classification Report:
              precision    recall  f1-score   support

    not_hate       0.76      1.00      0.87      1159
        hate       0.00      0.00      0.00       358

    accuracy                           0.76      1517
   macro avg       0.38      0.50      0.43      1517
weighted avg       0.58      0.76      0.66      1517


Task 3 (Explicit Language) Test Results:
Precision: 0.6854
Recall: 0.5049
weighted F1 Score: 0.5383
macro F1 Score: 0.4856

Classification Report:
              precision    recall  f1-score   support

not_explicit       0.81      0.46      0.59      1159
    explicit       0.27      0.66      0.39       358

    accuracy                           0.50      1517
   macro avg       0.54      0.56      0.49      1517
weighted avg       0.69      0.50      0.54      1517

